In [1]:
from wordcloud import WordCloud

# Manual PreProcessing (Lower Case All)

In [2]:
#make all lowercase
def lower_df_text(df, text_columns):
    for column in text_columns:
        df[column] = df[column].apply(lambda x: x.lower())
    return

# Generating Wordclouds

In [3]:
def plot_wordcloud(text, figname='wordcloud', rgb=(255,255,255)):
    font_path = "/Library/Fonts/DIN Condensed Bold.ttf"
    wc = WordCloud(background_color="white",max_words=25,
                   collocations=False, font_path=font_path, scale=5, color_func=lambda *args, **kwargs: rgb)
    wc.generate(text)
    
    fig, ax = plt.subplots(figsize=(14,18))
    plt.imshow(wc)
    plt.savefig('data/wordclouds/{}'.format(figname), dpi=240)


# Generating Network Graphs


In [4]:
import networkx as nx
from networkx.algorithms import community

In [5]:
# Functionize

def generate_graph(edge_filepath, node_filepath, title='gephi_graph'):
    edges = pd.read_csv(edge_filepath, header=None)
    nodes = pd.read_csv(node_filepath, header=None)
    edges.columns = ['Source', 'Target']
    nodes.columns = ['Node', 'Alias']
    nodes = nodes.astype('str')
    edges = edges.astype('str')
    edge_tuples = []
    for source, target in zip(edges['Source'], edges['Target']):
        edge_tuples.append((source, target))
    G = nx.DiGraph()
    G.add_nodes_from(nodes['Node'])
    G.add_edges_from(edge_tuples)
    print(nx.info(G))
    nx.write_gexf(G, edge_filepath.append('{}'.format(title)))
    

# Text Processing Pipeline (Too Slow)
from https://www.kaggle.com/balatmak/text-preprocessing-steps-and-universal-pipeline#Reusable-pipeline

In [6]:
import numpy as np
import multiprocessing as mp

import string
import spacy 
import en_core_web_sm
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator
from normalise import normalise

nlp = en_core_web_sm.load()


class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self,
                 variety="BrE",
                 user_abbrevs={},
                 n_jobs=1, custom_stop_words=[]):
        """
        Text preprocessing transformer includes steps:
            1. Text normalization
            2. Punctuation removal
            3. Stop words removal
            4. Lemmatization
        
        variety - format of date (AmE - american type, BrE - british format) 
        user_abbrevs - dict of user abbreviations mappings (from normalise package)
        n_jobs - parallel jobs to run
        """
        self.variety = variety
        self.user_abbrevs = user_abbrevs
        self.n_jobs = n_jobs
        self.custom_stop_words=custom_stop_words

    def fit(self, X, y=None):
        return self

    def transform(self, X, *_):
        X_copy = X.copy()

        partitions = 1
        cores = mp.cpu_count()
        if self.n_jobs <= -1:
            partitions = cores
        elif self.n_jobs <= 0:
            return X_copy.apply(self._preprocess_text)
        else:
            partitions = min(self.n_jobs, cores)

        data_split = np.array_split(X_copy, partitions)
        pool = mp.Pool(cores)
        data = pd.concat(pool.map(self._preprocess_part, data_split))
        pool.close()
        pool.join()

        return data

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        normalized_text = self._normalize(text)
        doc = nlp(normalized_text)
        removed_punct = self._remove_punct(doc)
        removed_stop_words = self._remove_stop_words(removed_punct)
        return self._lemmatize(removed_stop_words)

    def _normalize(self, text):
        # some issues in normalise package
        try:
            return ' '.join(normalise(text, variety=self.variety, user_abbrevs=self.user_abbrevs, verbose=False))
        except:
            return text

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if t.is_stop != True and t.text not in self.custom_stop_words  and 'https' not in t.text]

    def _lemmatize(self, doc):
        return ' '.join([t.lemma_ for t in doc])

/Users/jacobmullins/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.semi_supervised.label_propagation module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.semi_supervised. Anything that cannot be imported from sklearn.semi_supervised is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/jacobmullins/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelPropagation from version 0.18 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# Democratic Convention Data


In [67]:
dconvention_df = pd.read_csv('large_data/dconvention-tweet-ids.csv')
# nodes = 15,737
# edges = 88,537
# tweets = 134,687
# total users = 41,000


In [71]:
dconvention_df

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Fri Jul 29 07:28:18 +0000 2016,DemsInPhilly HillaryClinton,NaN,https://youtu.be/pnXiy4D_I8g,0,758927111146962945,NaN,NaN,NaN,...,111,293,6,London,Tchi Mbouani,tchimbouani,832,NaN,http://www.waterofpeace.blog,False
1,NaN,Mon Jul 25 05:59:10 +0000 2016,LoveAndKindness DemsInPhilly ImWithHer Clinton...,https://twitter.com/LoriSums/status/7574551296...,NaN,7,757455129687629824,NaN,NaN,NaN,...,7042,6064,210,NaN,Lori,LoriSums,482372,NaN,NaN,False
2,NaN,Wed Jul 27 00:34:54 +0000 2016,GMO DemsInPhilly DemConvention,NaN,NaN,2,758098299501023232,NaN,NaN,NaN,...,3670,2820,151,NYC,Karen K,kreativekonnect,139906,NaN,https://www.instagram.com/karkap617/,False
3,NaN,Fri Jul 29 03:25:26 +0000 2016,DemsInPhilly,NaN,NaN,6,758865994383106053,NaN,NaN,NaN,...,401,286,14,"Chicago, IL",Stephanie Hoeman,stephaniehoeman,5648,NaN,NaN,False
4,NaN,Mon Jul 25 05:50:31 +0000 2016,LoveAndKindness DemsInPhilly ImWithHer Clinton...,https://twitter.com/LoriSums/status/7574529512...,NaN,5,757452951203946496,NaN,NaN,NaN,...,7042,6064,210,NaN,Lori,LoriSums,482372,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134682,NaN,Tue Jul 26 23:17:11 +0000 2016,dnc,NaN,NaN,1,758078741285187585,NaN,NaN,NaN,...,1235,912,97,"Boulder, CO",Jann Scott TV,JannScott,17805,NaN,http://c1n.tv/JannScottLive/,False
134683,NaN,Thu Jul 28 03:40:08 +0000 2016,DemsInPhilly,NaN,NaN,0,758507305667334145,NaN,NaN,NaN,...,1153,1803,12,London,Jules Mason,1975jpm,12574,NaN,NaN,False
134684,NaN,Thu Jul 28 03:40:06 +0000 2016,DemsInPhilly DemConvention,NaN,NaN,0,758507297005932544,BarackObama,NaN,813286.0,...,3882,3653,185,"Silicon Valley, CA",Elizabeth #StayAwayFromCA Drescher,edrescherphd,13926,NaN,https://www.facebook.com/elizabethdrescherphd/,False
134685,NaN,Wed Jul 27 01:09:40 +0000 2016,NaN,NaN,https://twitter.com/auburnseminary/status/7579...,6,758107050283008000,NaN,NaN,NaN,...,2231,2430,78,NYC,Macky Alston,MackyAlston,4560,NaN,http://www.mackyalston.com,False


In [ ]:
dnodes_df = pd.read_csv('data/democrat/convention_mentions/dmention-nodes.csv')
dnodes_df
dnodes_df = dnodes_df.join(dcommunity_df, how='left', on='Id')
dconvention_df.merge(dnodes_df, left_on='user_screen_name', right_on='Label', how='outer')['modularity_class'].dropna()

# Republican Convention Data 

In [9]:
rconvention_df = pd.read_csv('large_data/rconvention-tweet-ids.csv')
rconvention_df.describe()
rconvention_df.columns

,favorite_count,id,in_reply_to_status_id,in_reply_to_user_id,retweet_count,reweet_id,retweet_screen_name,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count,user_time_zone
count,113631.000000,1.136310e+05,5.650000e+03,1.241000e+04,113631.000000,0.0,0.0,1.136310e+05,1.136310e+05,113631.000000,113631.000000,1.136310e+05,0.0
mean,3.855304,7.556725e+17,7.549740e+17,1.055017e+16,1.846283,NaN,NaN,3.264499e+04,1.316484e+04,2666.006187,150.543109,4.795156e+04,NaN
std,117.449660,4.707778e+14,1.442378e+16,8.699262e+16,52.296739,NaN,NaN,6.640411e+04,4.817132e+05,7370.316474,901.512807,8.528117e+04,NaN
min,0.000000,7.539204e+17,2.325725e+17,1.200000e+01,0.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,2.000000e+00,NaN
25%,0.000000,7.552497e+17,7.552227e+17,1.891643e+07,0.000000,NaN,NaN,2.916500e+03,3.950000e+02,545.000000,11.000000,7.076000e+03,NaN
50%,0.000000,7.555901e+17,7.555728e+17,4.006901e+07,0.000000,NaN,NaN,1.036500e+04,1.162000e+03,1188.000000,35.000000,1.965900e+04,NaN
75%,1.000000,7.560576e+17,7.559575e+17,2.541900e+08,0.000000,NaN,NaN,3.264450e+04,3.377000e+03,2717.000000,104.000000,5.337700e+04,NaN
max,26220.000000,7.564522e+17,7.564517e+17,7.562792e+17,11502.000000,NaN,NaN,1.053127e+06,7.967907e+07,508859.000000,116472.000000,1.611752e+06,NaN


Index(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'retweet_count', 'reweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_screen_name', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name.1',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified'],
      dtype='object')

# Democratic Candidates Data

# Republican Candidates Data

In [64]:
import preprocessor as p
p.clean('preprocessing and it’s always better to be sure that our data is cleaned 100%.')

'preprocessing and it’s always better to be sure that our data is cleaned %.'